In [3]:
using Pkg
Pkg.status() # Installed Packages

Status `C:\Users\Utsav\Utsav\Development\Julia\SpaceshipTitanic\Project.toml`
  [024491cd] BetaML v0.9.6
  [336ed68f] CSV v0.10.9
  [a93c6f00] DataFrames v1.5.0
  [31c24e10] Distributions v0.25.86
  [5789e2e9] FileIO v1.16.0
  [7073ff75] IJulia v1.24.0
  [82e4d734] ImageIO v0.6.6
  [6218d12a] ImageMagick v1.2.2
  [916415d5] Images v0.25.2
  [add582a8] MLJ v0.19.1
  [c6f25543] MLJDecisionTreeInterface v0.4.0
  [094fc8d1] MLJFlux v0.2.9
  [5ae90465] MLJScikitLearnInterface v0.3.0
  [91a5bcdd] Plots v1.38.8
  [d330b81b] PyPlot v2.11.1
  [321657f4] ScientificTypes v3.0.2
  [3646fa90] ScikitLearn v0.7.0
  [f3b207a7] StatsPlots v0.15.4
  [239c3e63] Vega v2.3.1
  [6385f0a0] WordCloud v0.10.8


In [4]:
# Necessary Imports

using Random
using CSV
using DataFrames
using Plots
using Plots.PlotMeasures
using StatsPlots
# using WordCloud
using Images, FileIO
using Vega
using Statistics
using MLJ
using BetaML
# using ScikitLearn
# using MLJDecisionTreeInterface

In [5]:
# Inputs

# Unnecessary Columns
UNNECESSARY_COLS = ["PassengerId","Name"]

# Handling Missing Value 
MISSING_STRATEGY = "median"
MISSING_COLS = ["Age", "FoodCourt", "ShoppingMall", "Spa", "VRDeck" ,"RoomService"]

# Handling Categorical Features
CATEGORICAL_COLS = ["HomePlanet", "CryoSleep","Cabin", "Destination" ,"VIP"]

5-element Vector{String}:
 "HomePlanet"
 "CryoSleep"
 "Cabin"
 "Destination"
 "VIP"

In [6]:
# Config

BASE_PATH = "../data"
TRAIN_PATH = "$BASE_PATH/train.csv"
TEST_PATH = "$BASE_PATH/test.csv"
SUB_PATH = "$BASE_PATH/sample_submission.csv"


"../data/sample_submission.csv"

In [7]:
train = CSV.File(TRAIN_PATH) |> DataFrame
test = CSV.File(TEST_PATH) |> DataFrame
sub = CSV.File(SUB_PATH) |> DataFrame
print("Done")

Done

In [8]:
function data_imputer_strategy(strategy)
    if (strategy == "mean") 
        return Statistics.mean
    elseif(strategy == "median")
        return Statistics.median
    elseif(strategy == "mode")
        return Statistics.mode
    end
end

data_imputer_strategy (generic function with 1 method)

In [9]:
TARGET = "Transported"
MISSING = "Z"
RANDOM_STATE = 42

42

In [10]:
function dropping_unnecessary_cols(X_train,X_test)
    for col in UNNECESSARY_COLS 
        if col in names(X_train)
            select!(X_train,Not(col))
        end
        if col in names(X_test)
            select!(X_test,Not(col))
        end
    end
    return X_train,X_test
end

function missing_value_handler(X_train,X_test,cols)    
    other_cols = filter(x -> ~(x in cols),names(X_train))
    simple_imputer = @load SimpleImputer pkg=BetaML
    imputer = simple_imputer(statistic=data_imputer_strategy(MISSING_STRATEGY))
    X_train2 = select(X_train,cols) |> MLJ.matrix
    X_test2 = select(X_test,cols) |> MLJ.matrix
    (fitResults,_,_) = MLJ.fit(imputer,0,X_train2)
    
    
    X_train_imputed = MLJ.transform(imputer,fitResults,X_train2) |> DataFrame
    X_test_imputed = MLJ.transform(imputer,fitResults,X_test2) |> DataFrame
    rename!(X_train_imputed,cols)
    rename!(X_test_imputed,cols)

    # for col in cols 
    #     X_train[!,col] = X_train_imputed[!,col]
    #     X_test[!,col] = X_test_imputed[!,col]
    # end
    
    X_train = DataFrame(hcat(select(X_train,other_cols) |> Matrix, X_train_imputed |> Matrix),:auto)
    X_test = DataFrame(hcat(select(X_test,other_cols) |> Matrix, X_test_imputed |> Matrix),:auto)
    rename!(X_train,[other_cols...,cols...])
    rename!(X_test,[other_cols...,cols...])


    X_train[!,cols] = convert.(Float64,X_train_imputed[!,cols])
    X_test[!,cols] = convert.(Float64,X_test_imputed[!,cols])

    return X_train,X_test
end

function categorical_feature_handler(X_train,X_test,cols)
    for col in cols 
        X_train[!,col] = replace!(X_train[!,col], missing => MISSING)
        X_test[!,col] = replace!(X_test[!,col], missing => MISSING)
    end  
    return X_train,X_test      
end

function label_encoding_handler(X_train,X_test,cols) 
    function gen_label_encoder() 
        label_encoder = Dict()
        for col in cols 
            col_mapper = Dict()
            idx = 0
            for val in unique(X_train[!,col])
                col_mapper[val] = idx
                idx+=1
            end
            label_encoder[col] = col_mapper
        end
        label_encoder
    end

    label_encoder = gen_label_encoder()

    for col in cols
        # X_train[!,col] = map(x -> (x===missing) ? missing : get(label_encoder[col],x,-1),X_train[!,col])
        # X_test[!,col] = map(x -> (x===missing) ? missing : get(label_encoder[col],x,-1),X_test[!,col])

        X_train[!,col] = map(x -> get(label_encoder[col],x,-1),X_train[!,col])
        X_test[!,col] = map(x -> get(label_encoder[col],x,-1),X_test[!,col])

    end
    
    X_train,X_test
end

function partition_data(y)
    fraction = 0.8
    rng = RANDOM_STATE
    shuffle = true

    train_idx, test_idx = MLJ.partition(eachindex(y[!,TARGET]), 0.8,shuffle=true,rng=RANDOM_STATE)

    train_idx, test_idx
end

function add_necessary_data_types(train,y_train,test) 
    map = []

    for col in names(train)
        if col in CATEGORICAL_COLS
            push!(map, col => Multiclass)
        else 
            push!(map, col => Continuous)
        end
    end

    # println(map...)
    
    train = coerce(train,map...)
    test = coerce(test,map...)
    # y_train = coerce(y_train,TARGET => Count)
    y_train = coerce(y_train,TARGET => Multiclass)
    train,y_train,test 
end

function preprocessor(train,test,sub) 
    X_train = select(train,Not(TARGET))
    y_train = select(train,TARGET)
    X_train,test = dropping_unnecessary_cols(X_train,test)
    X_train,test = missing_value_handler(X_train,test,MISSING_COLS)
    X_train,test = categorical_feature_handler(X_train,test,CATEGORICAL_COLS)
    X_train,test = label_encoding_handler(X_train,test,CATEGORICAL_COLS)
    X_train,y_train,test = add_necessary_data_types(X_train,y_train,test)
    # X_train,y_train,X_test,y_test = partition_data(X_train,y_train)
    # y_train = y_train[!,TARGET]

    X_train,y_train,test,sub
end

# X_train,y_train,X_test,y_test,test,sub = preprocessor(train,test,sub) 
X_train,y_train,test,sub = preprocessor(train,test,sub)
train_idx, test_idx = partition_data(y_train)
y_train = y_train[!,TARGET]
println(size(train_idx),size(test_idx),size(test),size(sub))
print("Done!")

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\Utsav\.julia\packages\MLJModels\OJDDo\src\loading.jl:159


import BetaML ✔


(6954,)(1739,)(4277, 11)(4277, 2)
Done!

In [50]:
sub

Row,PassengerId,Transported
,String7,Bool
1,0013_01,true
2,0018_01,false
3,0019_01,true
4,0021_01,true
5,0023_01,true
6,0027_01,true
7,0029_01,true
8,0032_01,true
9,0032_02,true


In [11]:
MLJ.models(matching(X_train,y_train))

4-element Vector{NamedTuple{(:name, :package_name, :is_supervised, :abstract_type, :deep_properties, :docstring, :fit_data_scitype, :human_name, :hyperparameter_ranges, :hyperparameter_types, :hyperparameters, :implemented_methods, :inverse_transform_scitype, :is_pure_julia, :is_wrapper, :iteration_parameter, :load_path, :package_license, :package_url, :package_uuid, :predict_scitype, :prediction_type, :reporting_operations, :reports_feature_importances, :supports_class_weights, :supports_online, :supports_training_losses, :supports_weights, :transform_scitype, :input_scitype, :target_scitype, :output_scitype)}}:
 (name = ConstantClassifier, package_name = MLJModels, ... )
 (name = DecisionTreeClassifier, package_name = BetaML, ... )
 (name = DeterministicConstantClassifier, package_name = MLJModels, ... )
 (name = RandomForestClassifier, package_name = BetaML, ... )

In [12]:
test_results=Dict()

Dict{Any, Any}()

In [13]:
# @doc MLJDecisionTreeInterface.DecisionTreeClassifier

Random.seed!(RANDOM_STATE)
model = @load DecisionTreeClassifier pkg=DecisionTree
model = machine(model(),X_train,y_train)
eval_results=evaluate!(model,rows=train_idx, resampling=CV(nfolds=10,shuffle=true,rng=RANDOM_STATE), measures=[MLJ.accuracy],operation=predict_mode)
y_hat = predict_mode(model, rows=test_idx)
test_results["DecisionTree"] = MLJ.accuracy(y_hat, y_train[test_idx])
eval_results

import MLJDecisionTreeInterface

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\Utsav\.julia\packages\MLJModels\OJDDo\src\loading.jl:159


 ✔

┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc DecisionTree.DecisionTreeClassifier` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{Table{Union{AbstractVector{Continuous}, AbstractVector{Multiclass{6561}}, AbstractVector{Multiclass{4}}, AbstractVector{Multiclass{3}}}}, AbstractVector{Multiclass{2}}}
│ 
│ fit_data_scitype(model) = Tuple{Table{<:Union{AbstractVector{<:Continuous}, AbstractVector{<:Coun

┌ Info: Creating subsamples from a subset of all rows. 
└ @ MLJBase C:\Users\Utsav\.julia\packages\MLJBase\WKVEo\src\resampling.jl:601


Evaluating over 10 folds:  20%[=====>                   ]  ETA: 0:00:38

Evaluating over 10 folds: 100%[=========================] Time: 0:00:10

PerformanceEvaluation object with these fields:
  measure, operation, measurement, per_fold,
  per_observation, fitted_params_per_fold,
  report_per_fold, train_test_rows
Extract:
┌────────────┬──────────────┬─────────────┬─────────┬───────────────────────────
│ measure    │ operation    │ measurement │ 1.96*SE │ per_fold                 ⋯
├────────────┼──────────────┼─────────────┼─────────┼───────────────────────────
│ Accuracy() │ predict_mode │ 0.717       │ 0.0179  │ [0.71, 0.721, 0.741, 0.7 ⋯
└────────────┴──────────────┴─────────────┴─────────┴───────────────────────────
                                                                1 column omitted


In [14]:
# @doc MLJDecisionTreeInterface.RandomForestClassifier

Random.seed!(RANDOM_STATE)
model = @load RandomForestClassifier pkg=DecisionTree
model = machine(model(),X_train,y_train)
eval_results=evaluate!(model,rows=train_idx, resampling=CV(nfolds=10,shuffle=true,rng=RANDOM_STATE), measures=[MLJ.accuracy],operation=predict_mode)
y_hat = predict_mode(model, rows=test_idx)
test_results["RandomForest"] =  MLJ.accuracy(y_hat, y_train[test_idx])
eval_results

import MLJDecisionTreeInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\Utsav\.julia\packages\MLJModels\OJDDo\src\loading.jl:159


┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc DecisionTree.RandomForestClassifier` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{Table{Union{AbstractVector{Continuous}, AbstractVector{Multiclass{6561}}, AbstractVector{Multiclass{4}}, AbstractVector{Multiclass{3}}}}, AbstractVector{Multiclass{2}}}
│ 
│ fit_data_scitype(model) = Tuple{Table{<:Union{AbstractVector{<:Continuous}, AbstractVector{<:Coun

Evaluating over 10 folds:  20%[=====>                   ]  ETA: 0:00:10

Evaluating over 10 folds:  30%[=======>                 ]  ETA: 0:00:07

...

Evaluating over 10 folds:  70%[=================>       ]  ETA: 0:00:02

Evaluating over 10 folds:  80%[====================>    ]  ETA: 0:00:02

Evaluating over 10 folds:  90%[======================>  ]  ETA: 0:00:01

Evaluating over 10 folds: 100%[=========================] Time: 0:00:07


PerformanceEvaluation object with these fields:
  measure, operation, measurement, per_fold,
  per_observation, fitted_params_per_fold,
  report_per_fold, train_test_rows
Extract:
┌────────────┬──────────────┬─────────────┬─────────┬───────────────────────────
│ measure    │ operation    │ measurement │ 1.96*SE │ per_fold                 ⋯
├────────────┼──────────────┼─────────────┼─────────┼───────────────────────────
│ Accuracy() │ predict_mode │ 0.784       │ 0.0146  │ [0.796, 0.793, 0.799, 0. ⋯
└────────────┴──────────────┴─────────────┴─────────┴───────────────────────────
                                                                1 column omitted


In [15]:
# @doc MLJScikitLearnInterface.ExtraTreesClassifier

Random.seed!(RANDOM_STATE)
model = @load ExtraTreesClassifier
model = machine(model(),X_train,y_train)
eval_results=evaluate!(model,rows=train_idx, resampling=CV(nfolds=10,shuffle=true,rng=RANDOM_STATE), measures=[MLJ.accuracy],operation=predict_mode)
y_hat = predict_mode(model, rows=test_idx)
test_results["ExtraTrees"] =  MLJ.accuracy(y_hat, y_train[test_idx])
eval_results

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\Utsav\.julia\packages\MLJModels\OJDDo\src\loading.jl:159


import MLJScikitLearnInterface ✔


┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc ScikitLearn.ExtraTreesClassifier` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{Table{Union{AbstractVector{Continuous}, AbstractVector{Multiclass{6561}}, AbstractVector{Multiclass{4}}, AbstractVector{Multiclass{3}}}}, AbstractVector{Multiclass{2}}}
│ 
│ fit_data_scitype(model) = Tuple{Table{<:AbstractVector{<:Continuous}}, AbstractVector{<:Finite}}
└ @

┌ Info: Creating subsamples from a subset of all rows. 
└ @ MLJBase C:\Users\Utsav\.julia\packages\MLJBase\WKVEo\src\resampling.jl:601


Evaluating over 10 folds:  20%[=====>                   ]  ETA: 0:00:17

...

Evaluating over 10 folds:  70%[=================>       ]  ETA: 0:00:03

Evaluating over 10 folds:  80%[====================>    ]  ETA: 0:00:02

Evaluating over 10 folds:  90%[======================>  ]  ETA: 0:00:01

Evaluating over 10 folds: 100%[=========================] Time: 0:00:08


PerformanceEvaluation object with these fields:
  measure, operation, measurement, per_fold,
  per_observation, fitted_params_per_fold,
  report_per_fold, train_test_rows
Extract:
┌────────────┬──────────────┬─────────────┬─────────┬───────────────────────────
│ measure    │ operation    │ measurement │ 1.96*SE │ per_fold                 ⋯
├────────────┼──────────────┼─────────────┼─────────┼───────────────────────────
│ Accuracy() │ predict_mode │ 0.779       │ 0.0142  │ [0.784, 0.792, 0.793, 0. ⋯
└────────────┴──────────────┴─────────────┴─────────┴───────────────────────────
                                                                1 column omitted


In [16]:
# @doc MLJScikitLearnInterface.LogisticClassifier

Random.seed!(RANDOM_STATE)
model = @load LogisticClassifier pkg=ScikitLearn
model = machine(model(),X_train,y_train)
eval_results=evaluate!(model,rows=train_idx, resampling=CV(nfolds=10,shuffle=true,rng=RANDOM_STATE), measures=[MLJ.accuracy],operation=predict_mode)
y_hat = predict_mode(model, rows=test_idx)
test_results["Logistic"] =  MLJ.accuracy(y_hat, y_train[test_idx])
eval_results

import MLJScikitLearnInterface

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\Utsav\.julia\packages\MLJModels\OJDDo\src\loading.jl:159


 ✔

┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc ScikitLearn.LogisticClassifier` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{Table{Union{AbstractVector{Continuous}, AbstractVector{Multiclass{6561}}, AbstractVector{Multiclass{4}}, AbstractVector{Multiclass{3}}}}, AbstractVector{Multiclass{2}}}
│ 
│ fit_data_scitype(model) = Tuple{Table{<:AbstractVector{<:Continuous}}, AbstractVector{<:Finite}}
└ @ M

C:\Users\Utsav\.julia\conda\3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Utsav\.julia\conda\3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

C:\Users\Utsav\.julia\conda\3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Utsav\.julia\conda\3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

C:\Users\Utsav\.julia\conda\3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Utsav\.julia\conda\3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

C:\Users\Utsav\.julia\conda\3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Utsav\.julia\conda\3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

C:\Users\Utsav\.julia\conda\3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
Evaluating over 10 folds: 100%[=========================] Time: 0:00:00


PerformanceEvaluation object with these fields:
  measure, operation, measurement, per_fold,
  per_observation, fitted_params_per_fold,
  report_per_fold, train_test_rows
Extract:
┌────────────┬──────────────┬─────────────┬─────────┬───────────────────────────
│ measure    │ operation    │ measurement │ 1.96*SE │ per_fold                 ⋯
├────────────┼──────────────┼─────────────┼─────────┼───────────────────────────
│ Accuracy() │ predict_mode │ 0.781       │ 0.0147  │ [0.802, 0.795, 0.774, 0. ⋯
└────────────┴──────────────┴─────────────┴─────────┴───────────────────────────
                                                                1 column omitted


In [17]:
# @doc MLJScikitLearnInterface.KNeighborsClassifier

Random.seed!(RANDOM_STATE)
model = @load KNeighborsClassifier
model = machine(model(),X_train,y_train)
eval_results=evaluate!(model,rows=train_idx, resampling=CV(nfolds=10,shuffle=true,rng=RANDOM_STATE), measures=[MLJ.accuracy],operation=predict_mode)
y_hat = predict_mode(model, rows=test_idx)
test_results["KNeighbors"] =  MLJ.accuracy(y_hat, y_train[test_idx])
eval_results

import MLJScikitLearnInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\Utsav\.julia\packages\MLJModels\OJDDo\src\loading.jl:159


┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc ScikitLearn.KNeighborsClassifier` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{Table{Union{AbstractVector{Continuous}, AbstractVector{Multiclass{6561}}, AbstractVector{Multiclass{4}}, AbstractVector{Multiclass{3}}}}, AbstractVector{Multiclass{2}}}
│ 
│ fit_data_scitype(model) = Tuple{Table{<:AbstractVector{<:Continuous}}, AbstractVector{<:Finite}}
└ @

┌ Info: Creating subsamples from a subset of all rows. 
└ @ MLJBase C:\Users\Utsav\.julia\packages\MLJBase\WKVEo\src\resampling.jl:601
Evaluating over 10 folds:  20%[=====>                   ]  ETA: 0:00:04

Evaluating over 10 folds:  40%[==========>              ]  ETA: 0:00:02

Evaluating over 10 folds:  70%[=================>       ]  ETA: 0:00:01

Evaluating over 10 folds: 100%[=========================] Time: 0:00:01


PerformanceEvaluation object with these fields:
  measure, operation, measurement, per_fold,
  per_observation, fitted_params_per_fold,
  report_per_fold, train_test_rows
Extract:
┌────────────┬──────────────┬─────────────┬─────────┬───────────────────────────
│ measure    │ operation    │ measurement │ 1.96*SE │ per_fold                 ⋯
├────────────┼──────────────┼─────────────┼─────────┼───────────────────────────
│ Accuracy() │ predict_mode │ 0.759       │ 0.0136  │ [0.741, 0.777, 0.76, 0.7 ⋯
└────────────┴──────────────┴─────────────┴─────────┴───────────────────────────
                                                                1 column omitted


In [18]:
# @doc MLJScikitLearnInterface.AdaBoostClassifier

Random.seed!(RANDOM_STATE)
model = @load AdaBoostClassifier
model = machine(model(),X_train,y_train)
eval_results=evaluate!(model,rows=train_idx, resampling=CV(nfolds=10,shuffle=true,rng=RANDOM_STATE), measures=[MLJ.accuracy],operation=predict_mode)
y_hat = predict_mode(model, rows=test_idx)
test_results["AdaBoost"] =  MLJ.accuracy(y_hat, y_train[test_idx])
eval_results

import MLJScikitLearnInterface ✔

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\Utsav\.julia\packages\MLJModels\OJDDo\src\loading.jl:159


┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc ScikitLearn.AdaBoostClassifier` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{Table{Union{AbstractVector{Continuous}, AbstractVector{Multiclass{6561}}, AbstractVector{Multiclass{4}}, AbstractVector{Multiclass{3}}}}, AbstractVector{Multiclass{2}}}
│ 
│ fit_data_scitype(model) = Tuple{Table{<:AbstractVector{<:Continuous}}, AbstractVector{<:Finite}}
└ @ M

┌ Info: Creating subsamples from a subset of all rows. 
└ @ MLJBase C:\Users\Utsav\.julia\packages\MLJBase\WKVEo\src\resampling.jl:601


Evaluating over 10 folds:  20%[=====>                   ]  ETA: 0:00:05

...

Evaluating over 10 folds:  70%[=================>       ]  ETA: 0:00:01

Evaluating over 10 folds:  80%[====================>    ]  ETA: 0:00:01

Evaluating over 10 folds:  90%[======================>  ]  ETA: 0:00:00

Evaluating over 10 folds: 100%[=========================] Time: 0:00:03


PerformanceEvaluation object with these fields:
  measure, operation, measurement, per_fold,
  per_observation, fitted_params_per_fold,
  report_per_fold, train_test_rows
Extract:
┌────────────┬──────────────┬─────────────┬─────────┬───────────────────────────
│ measure    │ operation    │ measurement │ 1.96*SE │ per_fold                 ⋯
├────────────┼──────────────┼─────────────┼─────────┼───────────────────────────
│ Accuracy() │ predict_mode │ 0.792       │ 0.0141  │ [0.784, 0.792, 0.818, 0. ⋯
└────────────┴──────────────┴─────────────┴─────────┴───────────────────────────
                                                                1 column omitted


In [19]:
# @doc MLJScikitLearnInterface.GradientBoostingClassifier

Random.seed!(RANDOM_STATE)
model = @load GradientBoostingClassifier
model = machine(model(),X_train,y_train)
eval_results=evaluate!(model,rows=train_idx, resampling=CV(nfolds=10,shuffle=true,rng=RANDOM_STATE), measures=[MLJ.accuracy],operation=predict_mode)
y_hat = predict_mode(model, rows=test_idx)
test_results["GradientBoosting"] =  MLJ.accuracy(y_hat, y_train[test_idx])
eval_results

import MLJScikitLearnInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\Utsav\.julia\packages\MLJModels\OJDDo\src\loading.jl:159


┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc ScikitLearn.GradientBoostingClassifier` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{Table{Union{AbstractVector{Continuous}, AbstractVector{Multiclass{6561}}, AbstractVector{Multiclass{4}}, AbstractVector{Multiclass{3}}}}, AbstractVector{Multiclass{2}}}
│ 
│ fit_data_scitype(model) = Tuple{Table{<:AbstractVector{<:Continuous}}, AbstractVector{<:Finite

C:\Users\Utsav\.julia\conda\3\lib\site-packages\sklearn\utils\deprecation.py:101: FutureWarning: Attribute `loss_` was deprecated in version 1.1 and will be removed in 1.3.
  warnings.warn(msg, category=FutureWarning)
C:\Users\Utsav\.julia\conda\3\lib\site-packages\sklearn\utils\deprecation.py:101: FutureWarning: Attribute `loss_` was deprecated in version 1.1 and will be removed in 1.3.
  warnings.warn(msg, category=FutureWarning)
Evaluating over 10 folds:  20%[=====>                   ]  ETA: 0:00:10

C:\Users\Utsav\.julia\conda\3\lib\site-packages\sklearn\utils\deprecation.py:101: FutureWarning: Attribute `loss_` was deprecated in version 1.1 and will be removed in 1.3.
  warnings.warn(msg, category=FutureWarning)
Evaluating over 10 folds:  30%[=======>                 ]  ETA: 0:00:08

...

C:\Users\Utsav\.julia\conda\3\lib\site-packages\sklearn\utils\deprecation.py:101: FutureWarning: Attribute `loss_` was deprecated in version 1.1 and will be removed in 1.3.
  warnings.warn(msg, category=FutureWarning)
Evaluating over 10 folds:  70%[=================>       ]  ETA: 0:00:03

C:\Users\Utsav\.julia\conda\3\lib\site-packages\sklearn\utils\deprecation.py:101: FutureWarning: Attribute `loss_` was deprecated in version 1.1 and will be removed in 1.3.
  warnings.warn(msg, category=FutureWarning)
Evaluating over 10 folds:  80%[====================>    ]  ETA: 0:00:02

C:\Users\Utsav\.julia\conda\3\lib\site-packages\sklearn\utils\deprecation.py:101: FutureWarning: Attribute `loss_` was deprecated in version 1.1 and will be removed in 1.3.
  warnings.warn(msg, category=FutureWarning)
Evaluating over 10 folds:  90%[======================>  ]  ETA: 0:00:01

C:\Users\Utsav\.julia\conda\3\lib\site-packages\sklearn\utils\deprecation.py:101: FutureWarning: Attribute `loss_` was deprecated in version 1.1 and will be removed in 1.3.
  warnings.warn(msg, category=FutureWarning)
Evaluating over 10 folds: 100%[=========================] Time: 0:00:09


PerformanceEvaluation object with these fields:
  measure, operation, measurement, per_fold,
  per_observation, fitted_params_per_fold,
  report_per_fold, train_test_rows
Extract:
┌────────────┬──────────────┬─────────────┬─────────┬───────────────────────────
│ measure    │ operation    │ measurement │ 1.96*SE │ per_fold                 ⋯
├────────────┼──────────────┼─────────────┼─────────┼───────────────────────────
│ Accuracy() │ predict_mode │ 0.795       │ 0.0155  │ [0.796, 0.797, 0.799, 0. ⋯
└────────────┴──────────────┴─────────────┴─────────┴───────────────────────────
                                                                1 column omitted


In [20]:
sort(collect(test_results),by = x->x[2])

7-element Vector{Pair{Any, Any}}:
     "DecisionTree" => 0.7303047728579644
       "KNeighbors" => 0.750431282346176
         "Logistic" => 0.7763082231167338
       "ExtraTrees" => 0.7918343875790684
         "AdaBoost" => 0.7935595169637722
     "RandomForest" => 0.7941345600920069
 "GradientBoosting" => 0.8050603795284647

In [21]:
#checking fitting parameters for possible best model
report(model)

In [22]:
y_sub = MLJ.predict(model,test)
y_sub

4277-element CategoricalDistributions.UnivariateFiniteVector{Multiclass{2}, Bool, UInt32, Float64}:
 UnivariateFinite{Multiclass{2}}(false=>0.365, true=>0.635)
 UnivariateFinite{Multiclass{2}}(false=>0.894, true=>0.106)
 UnivariateFinite{Multiclass{2}}(false=>0.115, true=>0.885)
 UnivariateFinite{Multiclass{2}}(false=>0.156, true=>0.844)
 UnivariateFinite{Multiclass{2}}(false=>0.472, true=>0.528)
 UnivariateFinite{Multiclass{2}}(false=>0.398, true=>0.602)
 UnivariateFinite{Multiclass{2}}(false=>0.115, true=>0.885)
 UnivariateFinite{Multiclass{2}}(false=>0.132, true=>0.868)
 UnivariateFinite{Multiclass{2}}(false=>0.115, true=>0.885)
 UnivariateFinite{Multiclass{2}}(false=>0.53, true=>0.47)
 ⋮
 UnivariateFinite{Multiclass{2}}(false=>0.495, true=>0.505)
 UnivariateFinite{Multiclass{2}}(false=>0.359, true=>0.641)
 UnivariateFinite{Multiclass{2}}(false=>0.0982, true=>0.902)
 UnivariateFinite{Multiclass{2}}(false=>0.437, true=>0.563)
 UnivariateFinite{Multiclass{2}}(false=>0.365, true=>0.635

In [24]:
# function load_model(name)
#     model = missing
#     if (name == "DecisionTree" )
#         model = @load DecisionTreeClassifier pkg=DecisionTree
#         # model = model()
#     elseif (name == "RandomForest")
#         model = @load RandomForestClassifier pkg=DecisionTree
#         # model = model()
#     elseif (name == "ExtraTrees")
#         model = @load ExtraTreesClassifier pkg=ScikitLearn
#         # model = model()
#     elseif (name == "Logistic")
#         model = @load LogisticClassifier pkg=ScikitLearn
#         # model = model()
#     elseif (name == "KNeighbors")
#         model = @load KNeighborsClassifier pkg=ScikitLearn
#         # model = model()
#     elseif (name == "AdaBoost")
#         model = @load AdaBoostClassifier pkg=ScikitLearn
#         # model = model()
#     elseif (name == "GradientBoosting")
#         model = @load GradientBoostingClassifier pkg=ScikitLearn
#         # model = model()
#     else 
#         throw(DomainError(name,"Invalid Model Name"))
#     end
#     model
# end

In [34]:
function load_model(name)
    model = missing
    if name == "DecisionTree" 
        model = @load DecisionTreeClassifier pkg=DecisionTree
        # model = model()
    elseif name == "RandomForest"
        model = @load RandomForestClassifier pkg=DecisionTree
        # model = model()
    elseif name == "ExtraTrees"
        model = @load ExtraTreesClassifier pkg=ScikitLearn
        # model = model()
    elseif name == "Logistic"
        model = @load LogisticClassifier pkg=ScikitLearn
        # model = model()
    elseif name == "KNeighbors"
        model = @load KNeighborsClassifier pkg=ScikitLearn
        # model = model()
    elseif name == "AdaBoost"
        model = @load AdaBoostClassifier pkg=ScikitLearn
        # model = model()
    elseif name == "GradientBoosting"
        model = @load GradientBoostingClassifier pkg=ScikitLearn
        # model = model()
    else 
        throw(DomainError(name,"Invalid Model Name"))
    end
    model
end

function setup_model(name,X,y)
    # model_preprocessing()
    model = load_model(name)
    model = machine(model(),X,y)
    model
end

setup_model (generic function with 1 method)

predict (generic function with 1 method)

In [58]:
function predict(model,test,sub)
    y_sub = MLJ.predict(model,test)
    sub[!,TARGET] = map(x -> x ? "True" : "False",pdf.(y_sub,true) .>= 0.5)
    sub
end

function evaluate(X_train,y_train,test,sub)
    train_results = Dict()
    test_results = Dict()

    all_models = [
        "DecisionTree",
        "RandomForest",
        "ExtraTrees",
        "Logistic",
        "KNeighbors",
        "AdaBoost",
        "GradientBoosting"
    ]

    # train_idx, test_idx = partition_data(y_train)
    # y_train = y_train[!,TARGET]

    for model_name in all_models 
        model = setup_model(model_name,X_train,y_train)
        eval_results=evaluate!(model,rows=train_idx, resampling=CV(nfolds=10,shuffle=true,rng=RANDOM_STATE), measures=[MLJ.accuracy],operation=predict_mode)
        test_results[model_name] = MLJ.accuracy(predict_mode(model, rows=test_idx), y_train[test_idx])
        train_results[model_name] = MLJ.accuracy(predict_mode(model, rows=train_idx), y_train[train_idx])
        println("$model_name 10 Folds :\n $(eval_results.per_fold)\n\n\n")
        sub = predict(model,test,sub)
        CSV.write("../output/$model_name.csv",sub)
    end

    train_results,test_results
end

train_results,test_results = evaluate(X_train,y_train,test,sub)

import MLJDecisionTreeInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\Utsav\.julia\packages\MLJModels\OJDDo\src\loading.jl:159
┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc DecisionTree.DecisionTreeClassifier` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{Table{Union{AbstractVector{Continuous}, AbstractVector{Multiclass{6561}}, AbstractVector{Multiclass{4}}, AbstractVector{Multiclass{3}}}}, Abstra

Evaluating over 10 folds:  30%[=======>                 ]  ETA: 0:00:00

Evaluating over 10 folds:  60%[===============>         ]  ETA: 0:00:00

Evaluating over 10 folds:  90%[======================>  ]  ETA: 0:00:00

...

C:\Users\Utsav\.julia\conda\3\lib\site-packages\sklearn\utils\deprecation.py:101: FutureWarning: Attribute `loss_` was deprecated in version 1.1 and will be removed in 1.3.
  warnings.warn(msg, category=FutureWarning)
Evaluating over 10 folds:  80%[====================>    ]  ETA: 0:00:02

C:\Users\Utsav\.julia\conda\3\lib\site-packages\sklearn\utils\deprecation.py:101: FutureWarning: Attribute `loss_` was deprecated in version 1.1 and will be removed in 1.3.
  warnings.warn(msg, category=FutureWarning)
Evaluating over 10 folds:  90%[======================>  ]  ETA: 0:00:01

C:\Users\Utsav\.julia\conda\3\lib\site-packages\sklearn\utils\deprecation.py:101: FutureWarning: Attribute `loss_` was deprecated in version 1.1 and will be removed in 1.3.
  warnings.warn(msg, category=FutureWarning)
Evaluating over 10 folds: 100%[=========================] Time: 0:00:08


GradientBoosting 10 Folds :
 [[0.7959770114942528, 0.7974137931034483, 0.7988505747126436, 0.7959770114942528, 0.8287769784172662, 0.8, 0.7741007194244605, 0.7870503597122303, 0.743884892086331, 0.823021582733813]]





(Dict{Any, Any}("RandomForest" => 0.9749784296807593, "KNeighbors" => 0.8143514524014955, "DecisionTree" => 0.9744032211676733, "ExtraTrees" => 0.9805867126833477, "AdaBoost" => 0.7952257693413862, "Logistic" => 0.7766752947943629, "GradientBoosting" => 0.8120506183491516), Dict{Any, Any}("RandomForest" => 0.7929844738355376, "KNeighbors" => 0.750431282346176, "DecisionTree" => 0.7366302472685451, "ExtraTrees" => 0.78953421506613, "AdaBoost" => 0.7935595169637722, "Logistic" => 0.7763082231167338, "GradientBoosting" => 0.8050603795284647))

In [59]:
sort(collect(test_results),by=x->x[2])

7-element Vector{Pair{Any, Any}}:
     "DecisionTree" => 0.7366302472685451
       "KNeighbors" => 0.750431282346176
         "Logistic" => 0.7763082231167338
       "ExtraTrees" => 0.78953421506613
     "RandomForest" => 0.7929844738355376
         "AdaBoost" => 0.7935595169637722
 "GradientBoosting" => 0.8050603795284647

In [60]:
sort(collect(train_results),by=x->x[2])

7-element Vector{Pair{Any, Any}}:
         "Logistic" => 0.7766752947943629
         "AdaBoost" => 0.7952257693413862
 "GradientBoosting" => 0.8120506183491516
       "KNeighbors" => 0.8143514524014955
     "DecisionTree" => 0.9744032211676733
     "RandomForest" => 0.9749784296807593
       "ExtraTrees" => 0.9805867126833477